### FUNCTIONS TO BE INVOKED IN METHODS FOR VICTOR

In [ ]:
def base_google_methods(path):
    
    from google.cloud import vision
    # Explicitly specify the credentials using a service account key file
    from google.oauth2 import service_account
    credentials = service_account.Credentials.from_service_account_file('/Users/jferro/Downloads/fast-pagoda-402720-33f7363eed6b.json')
    
    # Initialize the client with the specified credentials
    client = vision.ImageAnnotatorClient(credentials=credentials)

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    
    return client, image
    

# -------------------------------------------------------------------------------------


def detect_document(path):
    """Detects document features in an image."""
    client, image = base_google_methods(path)
    
    response = client.document_text_detection(image=image)

    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            print(f"\nBlock confidence: {block.confidence}\n")

            for paragraph in block.paragraphs:
                print("Paragraph confidence: {}".format(paragraph.confidence))

                for word in paragraph.words:
                    word_text = "".join([symbol.text for symbol in word.symbols])
                    print(
                        "Word text: {} (confidence: {})".format(
                            word_text, word.confidence
                        )
                    )

                    for symbol in word.symbols:
                        print(
                            "\tSymbol: {} (confidence: {})".format(
                                symbol.text, symbol.confidence
                            )
                        )

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    
    return response


# -------------------------------------------------------------------------------------


def detect_labels(path):
    """Detects labels in the file."""
    
    client, image = base_google_methods(path)

    response = client.label_detection(image=image)
    labels = response.label_annotations
    print("Labels:")

    for label in labels:
        print(label.description)

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    
    return response


# -------------------------------------------------------------------------------------


def detect_landmarks(path):
    """Detects landmarks in the file."""
    client, image = base_google_methods(path)
    
    response = client.landmark_detection(image=image)
    landmarks = response.landmark_annotations
    print("Landmarks:")

    for landmark in landmarks:
        print(landmark.description)
        for location in landmark.locations:
            lat_lng = location.lat_lng
            print(f"Latitude {lat_lng.latitude}")
            print(f"Longitude {lat_lng.longitude}")

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )

    return response


# -------------------------------------------------------------------------------------


def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    client, image = base_google_methods(path)

    objects = client.object_localization(image=image).localized_object_annotations

    print(f"Number of objects found: {len(objects)}")
    for object_ in objects:
        print(f"\n{object_.name} (confidence: {object_.score})")
        print("Normalized bounding polygon vertices: ")
        for vertex in object_.bounding_poly.normalized_vertices:
            print(f" - ({vertex.x}, {vertex.y})")
        
    return response


# -------------------------------------------------------------------------------------


def detect_safe_search(path):
    """Detects unsafe features in the file."""
    client, image = base_google_methods(path)

    response = client.safe_search_detection(image=image)
    safe = response.safe_search_annotation

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = (
        "UNKNOWN",
        "VERY_UNLIKELY",
        "UNLIKELY",
        "POSSIBLE",
        "LIKELY",
        "VERY_LIKELY",
    )
    print("Safe search:")

    print(f"adult: {likelihood_name[safe.adult]}")
    print(f"medical: {likelihood_name[safe.medical]}")
    print(f"spoofed: {likelihood_name[safe.spoof]}")
    print(f"violence: {likelihood_name[safe.violence]}")
    print(f"racy: {likelihood_name[safe.racy]}")

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )


# -------------------------------------------------------------------------------------


def detect_safe_search(path):
    """Detects unsafe features in the file."""
    client, image = base_google_methods(path)

    response = client.safe_search_detection(image=image)
    safe = response.safe_search_annotation

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = (
        "UNKNOWN",
        "VERY_UNLIKELY",
        "UNLIKELY",
        "POSSIBLE",
        "LIKELY",
        "VERY_LIKELY",
    )
    print("Safe search:")

    print(f"adult: {likelihood_name[safe.adult]}")
    print(f"medical: {likelihood_name[safe.medical]}")
    print(f"spoofed: {likelihood_name[safe.spoof]}")
    print(f"violence: {likelihood_name[safe.violence]}")
    print(f"racy: {likelihood_name[safe.racy]}")

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )

# -------------------------------------------------------------------------------------


def detect_web(path):
    """Detects web annotations given an image."""
    client, image = base_google_methods(path)

    response = client.web_detection(image=image)
    annotations = response.web_detection

    if annotations.best_guess_labels:
        for label in annotations.best_guess_labels:
            print(f"\nBest guess label: {label.label}")

    if annotations.pages_with_matching_images:
        print(
            "\n{} Pages with matching images found:".format(
                len(annotations.pages_with_matching_images)
            )
        )

        for page in annotations.pages_with_matching_images:
            print(f"\n\tPage url   : {page.url}")

            if page.full_matching_images:
                print(
                    "\t{} Full Matches found: ".format(len(page.full_matching_images))
                )

                for image in page.full_matching_images:
                    print(f"\t\tImage url  : {image.url}")

            if page.partial_matching_images:
                print(
                    "\t{} Partial Matches found: ".format(
                        len(page.partial_matching_images)
                    )
                )

                for image in page.partial_matching_images:
                    print(f"\t\tImage url  : {image.url}")

    if annotations.web_entities:
        print("\n{} Web entities found: ".format(len(annotations.web_entities)))

        for entity in annotations.web_entities:
            print(f"\n\tScore      : {entity.score}")
            print(f"\tDescription: {entity.description}")

    if annotations.visually_similar_images:
        print(
            "\n{} visually similar images found:\n".format(
                len(annotations.visually_similar_images)
            )
        )

        for image in annotations.visually_similar_images:
            print(f"\tImage url    : {image.url}")

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )

# -------------------------------------------------------------------------------------


# Call the function with the path to your image file
# response = detect_document("/Users/jferro/Downloads/hack.jpg")
# response = detect_labels("/Users/jferro/Downloads/hack.jpg")
# response = detect_landmarks('/Users/jferro/Downloads/statue.jpeg')
# response = localize_objects('/Users/jferro/Downloads/statue.jpeg')
# response = detect_safe_search('/Users/jferro/Downloads/statue.jpeg')
# response = detect_web('/Users/jferro/Downloads/statue.jpeg')


### CLASS FOR VICTOR

In [ ]:
class google_julen:

    # Method 1
    def detect_document_method(self, path):
        # Method code here
        return detect_document(path)
    
    # Method 1
    def detect_labels_method(self, path):
        # Method code here
        return detect_labels(path)
    
    # Method 1
    def detect_landmarks_method(self, path):
        # Method code here
        return detect_landmarks(path)
    
    # Method 1
    def localize_objects_method(self, path):
        # Method code here
        return localize_objects(path)
    
    # Method 1
    def detect_safe_search_method(self, path):
        # Method code here
        return detect_safe_search(path)

    # Method 1
    def detect_web_method(self, path):
        # Method code here
        return detect_web(path)
    

In [ ]:
# LIST OF METHODS
# response = detect_document_method("/Users/jferro/Downloads/hack.jpg")
# response = detect_labels_method("/Users/jferro/Downloads/hack.jpg")
# response = detect_landmarks_method('/Users/jferro/Downloads/statue.jpeg')
# response = localize_objects_method('/Users/jferro/Downloads/statue.jpeg')
# response = detect_safe_search_method('/Users/jferro/Downloads/statue.jpeg')
# response = detect_web_method('/Users/jferro/Downloads/statue.jpeg')

path = "/Users/jferro/Downloads/hack.jpg"

# google_julen().detect_document_method(path)
# google_julen().detect_labels_method(path)
# google_julen().detect_landmarks_method(path)
# google_julen().localize_objects_method(path)
# google_julen().detect_safe_search_method(path)
google_julen().detect_web_method(path)

### TESTING WITH A PDF

In [ ]:
from pdf2image import convert_from_path
import matplotlib.pyplot as plt

# Define the path to your PDF file
pdf_file = '/Users/jferro/Downloads/CS229_Lecture_Notes.pdf'

# Store PDF pages as images
images = convert_from_path(pdf_file)



In [ ]:
import os

# Create a directory to store the images if it doesn't exist
output_directory = 'pdf_images'
os.makedirs(output_directory, exist_ok=True)

# Save each image with the page number as the file name
for i, image in enumerate(images):
    # Create a file name using the page number
    file_name = os.path.join(output_directory, f'page_{i + 1}.png')
    
    # Save the image as a PNG file
    image.save(file_name, 'PNG')

# # Display each page in a plot (optional)
# for i, page_image in enumerate(images):
#     plt.figure()
#     plt.imshow(page_image)
#     plt.title(f"Page {i + 1}")
#     plt.show()

print(f"{len(images)} images saved in '{output_directory}' with page numbers as names.")


In [101]:

path = "/Users/jferro/Desktop/work/repos/KingsOfRecursion/research_julen/google_api/pdf_images/page_1.png"

# response = google_julen().detect_document_method(path)

# Give related words to the image
# google_julen().detect_labels_method(path)

# The place in which we are
# google_julen().detect_landmarks_method(path)

# google_julen().localize_objects_method(path)

# Quality assurance for VIOLENCE, RACY...
# google_julen().detect_safe_search_method(path)

google_julen().detect_web_method(path)


Best guess label: document

10 Web entities found: 

	Score      : 0.552299976348877
	Description: Machine learning

	Score      : 0.46950000524520874
	Description: Stochastic gradient descent

	Score      : 0.44589999318122864
	Description: Gradient descent

	Score      : 0.4244932234287262
	Description: Product design

	Score      : 0.3898000121116638
	Description: Gradient

	Score      : 0.380449503660202
	Description: Text

	Score      : 0.37560001015663147
	Description: Design

	Score      : 0.3704513609409332
	Description: Document

	Score      : 0.35850000381469727
	Description: Learning

	Score      : 0.3569999933242798
	Description: Data set

10 visually similar images found:

	Image url    : https://cdn.aglasem.com/aglasem-doc/9b809f4c-a771-11ec-8ed5-0a5e36bc6706/1.jpg
	Image url    : https://cdn.aglasem.com/aglasem-doc/e5deccbe-5e78-11eb-b617-02f21f5619c4/1.jpg
	Image url    : https://cdn.aglasem.com/aglasem-doc/e3fc7446-5e78-11eb-984c-02f21f5619c4/2.jpg
	Image url    : htt